<a href="https://colab.research.google.com/github/JuanAcevedo08/JuanAcevedo08/blob/main/documentado_organizado_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Importacion de datos

In [ ]:
#De GCP vamos a acceder donde se encuentran los datos desde la nube para poder cargalos
!wget --no-check-certificate https://storage.googleapis.com/platzi-tf2/sign-language-img.zip \-O /tmp/sign-language-img.zip

In [ ]:
import zipfile #Librería para manejar archivos zip
import os  #Poder acceder al sistema operativo para manejar rutas

In [ ]:
file_ = "/tmp/sign-language-img.zip" #Accedemos a la ruta donde se encuentra el archvio zip
zip_file = zipfile.ZipFile(file_) #Convertimos en un archivo zip lo que estaba en esa ruta
zip_file.extractall("/tmp/sign-language-img") #Extraemos todos los valores del archivo zip que acababamos de crear
zip_file.close() #Cerramos el archivo para no consumir mas espacio

# 2 Importacion de librerías y creacion de generadores



In [ ]:
import numpy as np #Uso de algebra lineal
import string #Manejo de strings
# Aprovechar mejor graficas
%matplotlib inline
import matplotlib.pyplot as plt #Graficar
import tensorflow as tf #Librería para uso de redes neuronales
image_generators = tf.keras.preprocessing.image.ImageDataGenerator #Generador de imagenes: Este se encarga de recorrer nuestras imagenes y pasarlas como entranamiento

In [ ]:
#Estructura del generador de entrenamiento
train_gen_est = image_generators(rescale=1/255) #Normalizo las imagenes , para que sea mucho mejor los resultados
test_gen_est = image_generators(rescale=1/255, validation_split=0.2) #Normalizo acá también y separo para el de validación

In [ ]:
#Rutas de donde saldrán las imágenes
train_path = '/tmp/sign-language-img/Train' #Donde saldrá para entrenar el modelo
test_path = '/tmp/sign-language-img/Test' #Acá saldrá para validar el modelo durante y después de acabar de entrenar

In [ ]:
                                        #Ajustar los generadores
#---- Generador de entrenamiento ----
train_gen = train_gen_est.flow_from_directory(
    train_path, #De dondé saldrán las imágenes
    target_size=(28, 28), #El tamaño de las imágenes (Importante que todas sean del mismo tamaño)
    color_mode='grayscale', #La escala de las imágenes
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    subset='training' #Que tipo de set de datos será este
)

#---- Generador de validación  ----
validation_gen = test_gen_est.flow_from_directory(
    test_path, #De dondé saldrán las imágenes
    target_size=(28, 28), #El tamaño de las imágenes (Importante que todas sean del mismo tamaño)
    color_mode='grayscale', #La escala de las imágenes
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    subset='validation' #Que tipo de set de datos será este
)

#---- Generador de test ----
test_gen = test_gen_est.flow_from_directory(
    test_path, #De dondé saldrán las imágenes
    target_size=(28, 28), #El tamaño de las imágenes (Importante que todas sean del mismo tamaño)
    color_mode='grayscale', #La escala de las imágenes
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    #Al definir train y validation sets el set de test ya se reconoce automaticamente
)

# 3 Funcion de graficacion de imagenes y mostrar primeras imagenes

In [ ]:
def plto_image(image):
  """
  Creo 5 columnas en 1 fila donde irán las imagenes , fijo también el tamaño que va a tener por imagen
  realizo un flaten en los axes para que sea mas facíl de graficar
  itero por cada imagen que se ponga y por cada ax para poder mostrar la iamgen con toda la fila y columna de la matriz diciendo que solo queremos el color en bloque 1
  retiro el axis para que sea mejor visiblemente
  realizo un tigh_layout para que no se sobrepongan algunas imagenes entre si
  finalmente muestro las imagenes creadas
  """
  fig, axes = plt.subplots(1, 5, figsize=(12, 10))
  axes = axes.flatten()
  for img, ax in zip(image, axes):
    ax.imshow(img[:, :, 0])
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
#Recorrer todas las imagenes del set de entrenamiento
image, _ = next(train_gen)
#Graficar las primeras 5 imagenes
plto_image(image[:5])

# 4 Clases

Esta parte es importante ya que serán la cantidad de neuronas que va a llevar nuestra red neuronal al momento de la capa final , como est capa final nos tiene que dar el valor de cada una de las clases para poder determinar cual de las es la elegida de esa manera para cualquier nn hay que poner las clases que tenemos y la longitud de la clase

ejemplo: tengo un trabajo de clasificar tipos de celular , tengo 8 clases pues las neuronas finales serían 8

In [ ]:
clases =  [char for char in string.ascii_uppercase if char != 'J' if char !='Z']
len(clases)

# 5 Graficador de resultados


In [ ]:
def plot_history(history):

  epochs = [v for v in range(20)]

  fig, ax = plt.subplots(1, 2)

  fig.set_size_inches(16, 10)

  train_acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']

  ax[0].plot(epochs, train_acc, 'g-o', label='Accuracy train')
  ax[0].plot(epochs, val_acc, 'r-o', label='Accuracy validation')
  ax[0].set_title('Accuracy Plots')
  ax[0].legend()
  ax[0].set_xlabel('Epochs')
  ax[0].set_ylabel('Accuracy')

  ax[1].plot(epochs, train_loss, 'g-o', label='Loss train')
  ax[1].plot(epochs, val_acc, 'r-o', label='Loss validation')
  ax[1].set_title('Loss Plots')
  ax[1].legend()
  ax[1].set_xlabel('Epochs')
  ax[1].set_ylabel('Loss')

  plt.show()

# 6 **REDES NEURONALES**

# 6.1 NN normal base


In [ ]:
def nn_basic():
  model_base = tf.keras.Sequential([
  #1. Capa inicial flatten con el indicador de tamaño
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  #2. Segunda capa con 256 neuronas y relu
  tf.keras.layers.Dense(256, activation='relu'),
  #3. Tercera capa con 128 neuronas y relu
  tf.keras.layers.Dense(128, activation='relu'),
  #4. Capa final con el valor de las clases para clasificar
  tf.keras.layers.Dense(len(clases), activation='softmax')
  ])
  return model_base

#Creacion del modelo base
model_base = nn_basic()
#Compilar el modelo para utilizar luego
model_base.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Mostrar estructura
model_base.summary()

# 6.2 NN con regularizadores y dropout



In [ ]:
regularizer = tf.keras.regularizers

In [ ]:
def nn_regularizer():
  model_regularizer = tf.keras.Sequential([
  #1. Iniciar con flatten para mejorar el proceso de la nn
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  #2. Capa incial que recibe entradas
  tf.keras.layers.Dense(256, kernel_regularizer=regularizer.l2(1e-5), activation='relu'),
  #3. Aplica dropout a la capa 2
  tf.keras.layers.Dropout(rate=0.2),
  #4. Capa que recibe la segunda capa como entrada
  tf.keras.layers.Dense(128, kernel_regularizer=regularizer.l2(1e-5), activation='relu'),
  #5. Aplica un dropout a la capa numero 4
  tf.keras.layers.Dropout(rate=0.2),
  #6. Finalmente recibe la informacion de las capas anteriores y devuelve las probabilidades de las clases para clasificar
  tf.keras.layers.Dense(len(clases), activation='softmax')
  ])

  return model_regularizer

#Crear el modelo
model_regularizer = nn_regularizer()
#Mostrar estructura
model_regularizer.summary()

# 6.3 CNN con regularizadores

In [ ]:
def cnn_model():
  model_cnn = tf.keras.Sequential([
  #1. Capa convolucional para extraer las caracteristicas principales
  tf.keras.layers.Conv2D(75, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  #2. Capa que se encarga de eliminar aquello que no está en las caracteristicas principales
  tf.keras.layers.MaxPool2D((2, 2)),
  #3. Capa para convertir los datos en vector para empezar a procesarlos en la red
  tf.keras.layers.Flatten(),
  #4. Primera capa de operaciones 256 neuronas
  tf.keras.layers.Dense(256, kernel_regularizer=regularizer.l2(1e-5), activation='relu'),
  #5. Capa de dropout para capa 4
  tf.keras.layers.Dropout(rate=0.2),
  #6. Capa de 128  neuronas
  tf.keras.layers.Dense(128, kernel_regularizer=regularizer.l2(1e-5), activation='relu'),
  #7. Capa de dropout para capa 6
  tf.keras.layers.Dropout(rate=0.2),
  #8. Capa final con los valores de 1las clases para clasificar
  tf.keras.layers.Dense(len(clases), activation='softmax')
  ])

  return model_cnn
#Crear el modelo
model_cnn = cnn_model()
#Compilar modelo para usarlo luego
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Mostrar estructura del modelo
model_cnn.summary()

# 7 **CALLBACKS**

# 7.0 Importacion de los callbacks necesarios

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, TensorBoard

# 7.1 Callbacks manuales

Callback para train
Para estos call back es importante conocer los metodos que tiene la clase padre para que esto pueda funcionar
Metodos[on_set_begin, on_epoch_begin, on_set_batch_begin, on_set_batch_end, on_epoch_end, on_set_end]
Donde set = (train, test, prediction)
### -on_set_begin = Momento donde empieza el set
### -on_epoch_begin = Momento donde empieza el primer epoch es decir la primera iteracion
### -on_set_batch_begin = Momento donde el lote del set empieza
### -on_set_batch_end = Momento donde el lote del set finaliza
### -on_epoch_end = Momento donde acaba la iteracion
### -on_set_end = Momento donde acaba de iterar el set

Así mismo para test y predicción

### -epoch = cantidad de iteraciones
### -logs = contiene los registros de cada iteracion para poder consultar
### -batch = lote a utilizar

In [ ]:
#Train callback
class TrainingCallback(Callback):
  def on_train_begin(self, logs=None):
    print('Starting training....')

  def on_epoch_begin(self, epoch, logs=None):
    print(f'Starting epoch {epoch}')

  def on_train_batch_begin(self, batch, logs=None):
    print(f'Training: Starting batch {batch}')
  def on_train_batch_end(self, batch, logs=None):
    print(f'Training: Finished batch {batch}')

  def on_epoch_end(self, epoch, logs=None):
    print(f'Finished epoch {epoch}')

  def on_train_end(self, logs={}):
     print('Finished training!')
     if logs.get('accuracy') >0.95:
      print("Lo logramos, nuestro modelo llego a 95%, detenemos el entrenamiento")
      self.model.stop_training = True
#Test Callback

class TestingCallback(Callback):
  def on_test_begin(self, logs=None):
    print('Starting testing....')

  def on_test_batch_begin(self, batch, logs=None):
    print(f'Testing: Starting batch {batch}')

  def on_test_batch_end(self, batch, logs=None):
    print(f'Testing: Finished batch {batch}')

  def on_test_end(self, logs=None):
     print(f'Finished testing!')

#Prediction - Validate Callback
class PredictionCallback(Callback):
  def on_predict_begin(self, logs=None):
    print('Prediction testing....')

  def on_predict_batch_begin(self, batch, logs=None):
    print(f'Prediction: Starting batch {batch}')

  def on_predict_batch_end(self, batch, logs=None):
    print(f'Prediction: Finished batch {batch}')

  def on_predict_end(self, logs=None):
     print('Finished prediction!')

# 7.2 Early Stopping

In [ ]:
early_callback = EarlyStopping(monitor='loss', patience=3, mode='auto')
#Monitor: Se refiere a la funcion a evaluar para no sobreajustar , ya sea perdida, accuracy etc
#Patience: La cantidad de iteraciones adicionales que va esperar para detenerse
#Mode: Depende de la funcion a monitorear lo que buscamos es minimizar o maximizar
#Ejemplo: Este empeiza las iteraciones , eleccionamos monitor: Accuracy y mode='auto' ,
#este va a iterar para alcanzar el accuracy mas alto cuando este ya no cambie va esperar 3 iteraciones mas, si este no cambia en esas 3 iteraciones , para el entrenamiento

# 7.3 ModelCheckPoint

In [ ]:
#Este se encarga de almacenar estructura y pesos de la red nueronl para poder volver a utilizarse , mostraré dos ejemplos de solo guardar pesos y otro de guardar todo el modelo
#1. Solo guardar los pesos
#Ruta donde se almacenan los pesos
weights_path = "model_weights/checkpoints.weights.h5"
checkpoint_weights = ModelCheckpoint(filepath=weights_path, save_weights_only=True, save_freq='epoch', verbose=1)
#filpath: La ruta donde va guardar los pesos
#save_weights_only: Solamente va guardar los pesos nada mas,
#save_freq: Cada cuanto va guardar los pesos en este caso cada epoch iteracion
#verbose: nos muestre lo que va haciendo
#2. Guardar todo el modelo
model_path = "model/best_model.h5"
checkpoint_model = ModelCheckpoint(filepath=model_path, save_best_only=True, save_weights_only=False, save_freq='epoch', verbose=1, monitor='val_accuracy')
#Nuevos cambios
#save_weights_only: Ahora en false para que guarde estructura y pesos
#save_best_inly: Solamente guarda los mejores
#monitor: De donde va clasificar el mejor eneste caso de val acurracy pero esto cambia dependiendo lo que el modelo necesite para almacenar el mejor modelo para n funcion

El modelo lo que hace para los pesos solamente
es iterar y en cada epoch es decir iteracion este va guardando en la ruta los pesos y nos va indicando el mensaje

Para el caso dos este itera y por cada iteracion va a guardar si solo si el modelo ve que el monitor es decir la funcion a monitoriar mejora , y este lo sobrecribe encaso de que encuentre un mejor modelo y lo guarda ahi

***como utilizar?***

Primero cargar unn modelo de donde va extraer esos valores

***PARA PESOS:***

crear un nuevo modelo (modelo).load_weights(cargar la ruta donde están los pesos guardados) de esa manera se cargan con los pesos del modelo anterior ya entrenado

***PARA MODELO COMPLETO:***

crear un nuevo modelo (modelo) = tf.keras.models.load_model(Ruta donde se guardó el mejor modelo)

***Otras maneras manuales de guardar pesos y modelo:***

Pesos:

model.save_weights(path)

Modelo:

model.save(path)

# 7.4 TensorBoard


In [ ]:
from time import time

In [ ]:
tensorboard_ = TensorBoard(log_dir=f"logs/{time()}")
#Log_dir= Directiro donde se almacenara el tensorboard
#Time() Para poder guardar la ingfomacion del modelo con la hora don tiempos de ejecucion y demas

**Implementacion**

una vez tenemos nuestro callback dentro del modelo , esté habrá sacado la infromacion
para cargarlo tendríamos que hacer :

%load_ext tensorboard

%tensorboard --logdir logs


# 7.5 Como implemenatr callbacks

esto se ve en la parte de ajustar las redes neuronales

.fit(
  callbacks=[El callback que desean implementar]
)

Mas adelante se implementará un modelo con todo para tener una referencia lo unico que cambiaría sería el callback a utilizar

# 8 Guardar solamente la estructura de un modelo

In [ ]:
#Para el ejemplo utilizaré el modelo de Convolucion para implementarlo
model_estructure = model_cnn.get_config()
#Implementar la estructura en otro modelo
model_with_estructure = tf.keras.Sequential.from_config(model_estructure)
#Revisar la estructura de este
model_with_estructure.summary()

# 9 Autotuner

El autotuner es una tecnica para buscar los hyperparametros dentro de las redes neuronales como lo son las neuronas o los learning rates , en este caso tenemos que lo que hacen es probar una cantidad de epocas con una combinacion y luego van probando otra hasta quedarse con la mejor combinacion
cuando usamos hyperband este incluye el factor= n , donde se va encargar de usar menos recursos para encontrar esto, esto funciona porque agarra la cantidad de epocas y las divide entre el numero del factor dando una nueva catnidad de epocas , y entrena esa cantidad de epocas con diferentes combinaciones , se queda con 1/n_factor de esas combinaciones les da mas recursos para que se entrenen y leugo vuelve y se queda con 1/n_factor de esos anteriores , para seleccionar la mejor combinacion

In [ ]:
#Importar la librería para usar el autotuner
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
#Funcion con el modelo con las posibles combinaciones
def cnn_tuner(hp):
  model_ = tf.keras.Sequential()
  #Capas iniciales de limpieza y filtración
  model_.add(tf.keras.layers.Conv2D(75, (3, 3), input_shape=(28, 28, 1), activation='relu'))
  model_.add(tf.keras.layers.MaxPool2D((2, 2)))
  model_.add(tf.keras.layers.Flatten())

  #Combinaciones posbiles
  hp_combinator = hp.Int('units', min_value=64, max_value=512, step=32)

  #Capa de cambio
  model_.add(tf.keras.layers.Dense(units=hp_combinator, activation='relu', kernel_regularizer=regularizer.l2(1e-5)))
  model_.add(tf.keras.layers.Dropout(rate=0.2)) #Dropout para la capa previa

  #Capas finales
  model_.add(tf.keras.layers.Dense(128, kernel_regularizer=regularizer.l2(1e-5), activation='relu')) #(Probar combinaciones opcionales en esta capa, en este caso por recursos no)
  model_.add(tf.keras.layers.Dropout(rate=0.2)) #Dropout para la capa previa
  model_.add(tf.keras.layers.Dense(len(clases), activation='softmax'))

  #Combinaciones posibles del Learning Rate
  lr_combinator = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  #Compilar el modelo y especificar que el optimizer es un combinador
  model_.compile(optimizer=tf.keras.optimizers.Adam(lr_combinator), loss='categorical_crossentropy', metrics=['accuracy'])

  return model_

In [ ]:
hyper_band = kt.Hyperband(
    cnn_tuner, #Modelo donde se encuentran los parametros de combinaciones
    objective='val_accuracy', #Sacar el mejor modelo respecto a la referencia que el proyecto necesite
    max_epochs=20, #Cantidad maxima de epocas
    factor=3, #Veces que va a probar diferentes epocas y combinaciones (Optimzar recursos)
    directory='combinations/', #Donde va guardar los modelos
    project_name='Tuner' #Nombre del proyecto
)
#Como entrnarlo
#tuner.search(datos_train, epochs=epochs_puestos, validation_data=datos_validation)
#Extraer mejore resultados: best_hps = tuner.get_best_hyperparameters(num_trails= 1)[0] #el numnero de trails es la cantidad de veces que va validar que los resultados finales sean correctos
#instanciar modelo con la best_combination
#hyper_model = tuner.hypermodel.build(best_hps) y entrenar con fit como se hace normal

# 10 Transfer Learning


Para el transfer learning es muy importante revisar los modelos los cuales vamos a usar y el que mejor se ajuste a nosotros ya que hay qeu cambiar los generadores con los cuales se entrenan los modelos , esto para que sea comatibles

El tl consiste en cortar hasta cierto punto una estrctura de red neuronal con los pesos de esa red congelados para aprovechar los epsos y la estructura de una manera que generalice , cortamos a una parte para qeu no sea tan especfica e implementamos las capas finales con nuestras clases

# 10.1 Nuevos generadores

In [ ]:
                                        #Ajustar los generadores para el moelo preentrenado
#Cambios para este
#Reajustar tamaño de imagenes para que sea compatible con el modelo preentrenado y reajustar la escala de colores con la que la escala del modelo preentrenado fue entrenado
#Cambiar nombre de genrador a train_gen_pt -> pt -> PreenTrenado

#---- Generador de entrenamiento ----
train_gen_pt = train_gen_est.flow_from_directory(
    train_path, #De dondé saldrán las imágenes
    target_size=(150, 150), #El tamaño de las imágenes que nos dice el modelo PREENTRENADO
    color_mode='rgb', #La escala de las imagenes que nuestro modelo preentrenado nos indica
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    subset='training' #Que tipo de set de datos será este
)

#---- Generador de validación  ----
validation_gen_pt = test_gen_est.flow_from_directory(
    test_path, #De dondé saldrán las imágenes
    target_size=(150, 150), #El tamaño de las imágenes que nos dice el modelo PREENTRENADO
    color_mode='rgb', #La escala de las imagenes que nuestro modelo preentrenado nos indica
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    subset='validation' #Que tipo de set de datos será este
)

#---- Generador de test ----
test_gen_pt = test_gen_est.flow_from_directory(
    test_path, #De dondé saldrán las imágenes
    target_size=(150, 150), #El tamaño de las imágenes que nos dice el modelo PREENTRENADO
    color_mode='rgb', #La escala de las imagenes que nuestro modelo preentrenado nos indica
    class_mode='categorical', #Tipo de clase que hay
    batch_size=(128), #Tamaño del lote en como va enviar las imágenes
    #Al definir train y validation sets el set de test ya se reconoce automaticamente
)

# 10.2 Keras Preentrenado Model (Inception_resnet_v2)

Este modelo se estuvo revisando para poder ajustarlo y ver como optimo para este ejemplo con nuestro ejemplo de lenguaje de señas

link de keras con mas pt_models =

[Keras Applications](https://keras.io/api/applications/)




In [ ]:
pt_model_keras = tf.keras.applications.inception_v3.InceptionV3 #Modelo preentrenado
pt_model = pt_model_keras(include_top=False, input_tensor=tf.keras.layers.Input(shape=(150, 150, 3)))
#Include_top: Al momento de crear el modelo especficamos que no queremos las ultimas capas ya que contienen las clases de ese modelo no del nestro
#Input_tensor: Importante ya que le indicamos que la primera capa van a ser nuestras entradas con ciertos tamaños que son los que tiene los generadores

In [ ]:
#Asignamos no entrenamiento a las capas ya que estas ya están entrenadas y solo queremos usar los pesos
for layer in pt_model.layers:
  layer.trainable = False

#Revisamos estructura de la red neuronal como la tenemos
pt_model.summary()

In [ ]:
#Cortar la red hasta un punto donde no sea tan especficio respecto a lo que hizo (Depende como lo necesites)
last_layer = pt_model.get_layer('mixed7')
last_ouput = last_layer.output #Nos da las capas para usar hasta mixed 7

In [ ]:
#Crear la Red con las capas previas y con las capas nuestras finales
#API Funcional
x = tf.keras.layers.Flatten()(last_ouput) #Flatten despues de mixed7
x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizer.l2(1e-5))(x)#Agregamos nueva capa casi llegando al final
x = tf.keras.layers.Dropout(rate=0.2)(x) #Dropout para la anterior
x = tf.keras.layers.Dense(len(clases), activation='softmax')(x) #Capa final con nuestras clases

#Crear nuestro modelo con la red neuronal que acabamos de armar
model_pt = tf.keras.Model(pt_model.input, x) #Especificar la entrada y la red o la salida que tiene
#Rectificar que la red haya quedado como queriamos
model_pt.summary()

Entrenar el modelo preentrenado

history = model_pt.fit(train_data(RESIZE), callbacks=[callback] , epochs=20, validation_data=validation_data(RESIZE))

# 11 Entrenamiento de modelos basicoss para ver como entrenar y visualizar

In [ ]:
#Modelo base creado previamente
#En historypara almacenar los datos y graficar resultados
history_base = model_base.fit(
    train_gen,
    epochs=20,
    callbacks=[early_callback],
    validation_data = validation_gen
)

In [ ]:
#Graficar resultados
plot_history(history_base)

In [ ]:
#Modelo convolucional creado previamente
hisstory_conv = model_cnn.fit(
    train_gen,
    epochs=20,
    callbacks=[tensorboard_],
    validation_data=validation_gen
)

In [ ]:
plot_history(hisstory_conv)

In [ ]:
#Usar tensorboard para ver el modelo
%load_ext tensorboard
%tensorboard --logdir logs
